In [1]:
import Enum_data as ed
import general_functions as gf
import operational_functions as of
import monthly_report_functions as mf
import income_statement_functions as isf
import others_functions as otf
import overhead_functions as ovf
import quantity_turnover_functions as qf
import raw_material_functions as rf
import cogs_functions as cf
import Cost_functions as cof

In [2]:
def run_program(symbol: str, year: int = 1400):
    report_types = [value.value for value in ed.names]
    all_reports = {}

    financial_years=gf.get_financial_years(symbol, year)
    for year in financial_years:
        parse_date = gf.parse_date_persian(year)
        dummy_dict = {}
        for report in report_types:
            report_type = ed.names[report].value
            sheet_num = ed.sheets[report_type].value
            table = ed.tabels[report_type].value
            reports = gf.get_results(symbol, parse_date, report_type, sheet_num)
            dummy_dict[report_type] = reports
        all_reports[year] = dummy_dict
    return all_reports

In [3]:
symbol = 'فخوز'
all_reports = run_program(symbol)

In [ ]:
url = all_reports['14031230']['RawMaterial'][14030930]['url']
date = 14030930
publish = 14031020
period = 9
rf.creat_raw_material_dataframe(symbol, url, date, period, publish)['report_this_year']

In [4]:
#Excel Format data
filtered_all_reports = {}
for fin_year in sorted(list(all_reports.keys()), reverse=True):
    filtered_all_reports[fin_year] = {}
    for report_type in [value.value for value in ed.names]:
        if report_type in ['Monthly_report']: continue
        if fin_year in all_reports.keys():
            if all_reports[fin_year][report_type][list(all_reports[fin_year][report_type].keys())[-1]]['period'] == 12:
                filtered_all_reports[fin_year][report_type] = all_reports[fin_year][report_type][list(all_reports[fin_year][report_type].keys())[-1]]
            else:
                filtered_all_reports[fin_year][report_type] = all_reports[fin_year][report_type][list(all_reports[fin_year][report_type].keys())[-1]]

In [89]:
all_data = {}
for year, report_types in filtered_all_reports.items():
    for report_type, st_dic in report_types.items():
        if report_type == "Monthly_report":
            continue

        date = gf.extract_date(st_dic["title"])
        period = st_dic["period"]
        publish = st_dic["publish"]
        url = st_dic["url"]

        if report_type == 'Operational':
            continue
            d = of.create_operational_dataframe(symbol, url, date, period, publish)
            all_data = sort_df_dic(all_data, report_type, d)
        elif report_type == 'RawMaterial' :
            continue
            d = rf.creat_raw_material_dataframe(symbol, url, date, period, publish)
            all_data = gf.sort_df_dic(all_data, report_type, d)
        elif report_type == 'COGS' :
            continue
            d = cf.create_cogs_dataframe(symbol, url, date, period, publish)
            all_data = gf.sort_df_dic(all_data,report_type, d)
        elif report_type == 'Cost' :
           continue
           d = cof.creat_Cost_dataframe(symbol, url, date, period, publish)
           all_data = gf.sort_df_dic(all_data , report_type , d)
        elif report_type == 'Incoeme_Statment' :
            continue
            d = isf.create_Incoeme_Statment_dataframe(symbol, url, date, period, publish)
            all_data =gf.sort_df_dic(all_data , report_type , d)
        elif report_type == 'Others' :
            continue
            d = otf.create_Others_dataframe(symbol, url, date, period, publish)
            all_data = gf.sort_df_dic(all_data, report_type , d)
        elif report_type == 'Overhead' :
            d = ovf.creat_Overhead_dataframe(symbol, url, date, period, publish)
            all_data = gf.sort_df_dic(all_data, report_type, d)
        

In [91]:
all_data['Overhead'].keys()

dict_keys([14021229, 14031230, 14011229, 14001229])

In [10]:
all_data = {}
for year, report_types in filtered_all_reports.items():
    for report_type, st_dic in report_types.items(): 
        date = extract_date(st_dic['title'])
        period = st_dic['period']
        publish = st_dic['publish']
        url = st_dic['url']

        if report_type == 'Monthly_report' :
            #mf.create_Monthly_report_dataframe(symbol, url, date, publish)
            continue
        if report_type == 'Operational':
            #d = of.create_operational_dataframe(symbol, url, date, period, publish)
            #all_data = sort_df_dic(all_data, report_type, d)
            continue
        if report_type == 'COGS' :
            #print(cf.create_cogs_dataframe(symbol, url, date, period, publish))
            continue
        if report_type == 'Cost' :
            #print(cof.creat_Cost_dataframe(symbol, url, date, period, publish))
            continue
        if report_type == 'Incoeme_Statment' :
            #print(isf.create_Incoeme_Statment_dataframe(symbol, url, date, period, publish))
            continue
        if report_type == 'Others' :
            #print(otf.create_Others_dataframe(symbol, url, date, period, publish))
            continue
        if report_type == 'Overhead' :
            #print(ovf.creat_Overhead_dataframe(symbol, url, date, period, publish))
            continue
        if report_type == 'RawMaterial' :
            break
            #print(rf.creat_raw_material_dataframe(symbol, url, date, period, publish))
            continue
        if report_type == 'Quantity_Turnover' :
            #print(qf.create_Quantity_Turnover_dataframe(symbol, url, date, period, publish))
            continue

In [113]:
import requests
import json
import re
import polars as pl
import Enum_data as ed
import general_functions as gf
from persiantools import characters, digits

In [125]:
def get_table(url: str, table: int):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:143.0) Gecko/20100101 Firefox/143.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Origin': 'https://www.codal.ir',
    'Connection': 'keep-alive',
    'Referer': 'https://www.codal.ir/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'Cookie': 'TS018fb0f7=01f9930bd2e2675d04882f623c888052df60031e7775d8e3c459dc4cb96bf8e870e0a9bf0e0eedff68bcce23fd9701c7a7fc0855c4; Unknown=1076170924.20480.0000'
    }
    response = requests.request("GET", url)
    statement = response.text
    pattern = r"var datasource = (.*?});"
    match = re.search(pattern, statement)
    if match:
        text = match.group(1)
    records = []
    records.append(
        (statement, text))
    for _, data in records:
        continue
    items = json.loads(data)['sheets']

    if isinstance(table, list):
        cells = []
        for t in table:
            raw_cells = items[0]['tables'][t]['cells']
            cells.append([(i['columnSequence'], i['rowSequence'], i['value'], i['periodEndToDate']) for i in raw_cells])
        return [x for xs in cells for x in xs]
    
    cells = items[0]['tables'][table]['cells']
    return [(i['columnSequence'], i['rowSequence'], i['value'], i['periodEndToDate']) for i in cells]

def create_dict_dataframes(url: str, date: int, report_type: str) -> dict:
    all_data ={ 'report_last_year' : pl.DataFrame(),
                'report_this_year': pl.DataFrame(),
                'est_remain': pl.DataFrame(),
                'est_next_year': pl.DataFrame()}
    
    cells_tuples = get_table(url, 6)
    return cells_tuples
    dates = sorted(list(set([i[-1] for i in cells_tuples if i[-1] != ''])))
    for date_ in dates:
        filtered_cells = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == '' or i[-1] == date_]
        df = pl.from_records(filtered_cells, schema=["col", "row", "value"], orient="row")
        df = df.pivot(values="value", on="col", index="row").sort("row")
        for i, col in enumerate(df.columns):
            value = df[col][1]        # value from first row
            if(isinstance(value, str)):
                for val in value.split():
                    if'/' in val:
                        num = int(val.replace('/', ''))
                        if num < date :
                            all_data['report_last_year'] = df
                            #all_data['report_last_year'] = all_data['report_last_year'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))
                        if num == date :
                            all_data['report_this_year'] = df
                            #all_data['report_this_year'] = all_data['report_this_year'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))
                        else:
                            if(val[:4] == str(date)[:4]):
                                all_data['est_remain'] = df
                                #all_data['est_remain'] = all_data['est_remain'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))                          
                                
                                all_data['est_next_year'] = df
                                #all_data['est_next_year'] = all_data['est_next_year'].insert_column(-1, pl.lit(int(digits.fa_to_en(date_).replace('/', ''))).alias("Date"))
    for key, value in all_data.items():
        if(value.is_empty()):
            all_data[key] = None  
    return all_data

def creat_Overhead_dataframe(symbol: str, url: str, date: int, period: int, publish: int) -> dict:
    all_data = create_dict_dataframes(url, date, 'Overhead')
    return all_data
    col_name = ['description',
                'Wages and Salaries Expense',
                'Depreciation Expense',
                'Energy Expense (Water, Electricity, Gas, and Fuel)',
                'Consumable Materials Expense',
                'Advertising Expense',
                'Sales Commission and Brokerage Fee',
                'After-Sales Service Expense',
                'Doubtful Accounts Expense / Bad Debt Expense',
                'Transportation and Delivery Expense',
                'Other Expenses']
            
    for key, data in all_data.items():
        if data is None: continue
        data = data.transpose(include_header=False)
        data = data.drop(data.columns[12])
        data = data[2]
        table_date = [d for d in str(data.row(0)[1]).split() if '/' in d][0]
        data = data.drop(data.columns[1])
        data = data.rename(dict(zip(data.columns, col_name)))
        data = data.insert_column(0, pl.lit(symbol).alias("Symbol"))
        data = data.insert_column(1, pl.lit(int(digits.fa_to_en(table_date).replace('/', ''))).alias("Date"))
        data = data.insert_column(2, pl.lit(period).alias("Period"))
        data = data.insert_column(3, pl.lit(publish).alias("Publish"))
        all_data[key] = data 

    return all_data     

In [101]:
date = 14040631
period =  6
publish = 14040824
url ='https://codal.ir/Reports/Decision.aspx?LetterSerial=A6GHwKf82I6VNWgrotzj7A%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=22'

In [126]:
creat_Overhead_dataframe(symbol, url, date, period, publish)

[(1, 1, 'شرح', ''),
 (1, 3, 'هزینه حقوق و دستمزد', ''),
 (1, 4, 'هزینه استهلاک', ''),
 (2, 1, 'هزینه سربار', '1403/12/30'),
 (2, 2, 'سال مالی منتهی به 1403/12/30', '1403/12/30'),
 (2, 3, '57972543', '1403/12/30'),
 (2, 4, '14536942', '1403/12/30'),
 (3, 2, 'دوره 6 ماهه منتهی به 1404/06/31', '1404/06/31'),
 (3, 3, '36512382', '1404/06/31'),
 (3, 4, '7340982', '1404/06/31'),
 (4, 2, 'برآورد دوره 6 ماهه منتهی به 1404/12/29', '1404/12/29'),
 (4, 3, '35232829', '1404/12/29'),
 (4, 4, '6860660', '1404/12/29'),
 (5, 2, 'برآورد سال مالی منتهی به 1404/12/29', '1404/12/29'),
 (5, 3, '0', '1404/12/29'),
 (5, 4, '0', '1404/12/29'),
 (6, 1, 'هزینه های اداری، عمومی و فروش', '1403/12/30'),
 (6, 2, 'سال مالی منتهی به 1403/12/30', '1403/12/30'),
 (6, 3, '16927352', '1403/12/30'),
 (6, 4, '1114023', '1403/12/30'),
 (7, 2, 'دوره 6 ماهه منتهی به 1404/06/31', '1404/06/31'),
 (7, 3, '11174489', '1404/06/31'),
 (7, 4, '521139', '1404/06/31'),
 (8, 2, 'برآورد دوره 6 ماهه منتهی به 1404/12/29', '1404/12/29'),
 

In [ ]:
test = {}
for year, report_dic in all_data.items():
    for name, df in report_dic.items():
        if df is None : continue
        if df['Date'][0] not in test:
            test[df['Date'][0]] = df
test


{14031230: shape: (7, 10)
 ┌────────┬──────────┬────────┬──────────┬───┬─────────┬────────────┬─────────┬───────────┐
 │ Symbol ┆ Date     ┆ Period ┆ Publish  ┆ … ┆ Type    ┆ Production ┆ Sales   ┆ Price     │
 │ ---    ┆ ---      ┆ ---    ┆ ---      ┆   ┆ ---     ┆ ---        ┆ ---     ┆ ---       │
 │ str    ┆ i32      ┆ i32    ┆ i32      ┆   ┆ str     ┆ i64        ┆ i64     ┆ i64       │
 ╞════════╪══════════╪════════╪══════════╪═══╪═════════╪════════════╪═════════╪═══════════╡
 │ فخوز   ┆ 14031230 ┆ 12     ┆ 14040824 ┆ … ┆ داخلی   ┆ 4445096    ┆ 555165  ┆ 129478816 │
 │ فخوز   ┆ 14031230 ┆ 12     ┆ 14040824 ┆ … ┆ داخلی   ┆ 967578     ┆ 858953  ┆ 240694072 │
 │ فخوز   ┆ 14031230 ┆ 12     ┆ 14040824 ┆ … ┆ داخلی   ┆ 2050460    ┆ 1329807 ┆ 234611990 │
 │ فخوز   ┆ 14031230 ┆ 12     ┆ 14040824 ┆ … ┆ داخلی   ┆ 5327975    ┆ 15661   ┆ 54727157  │
 │ فخوز   ┆ 14031230 ┆ 12     ┆ 14040824 ┆ … ┆ صادراتی ┆ 0          ┆ 103361  ┆ 262062674 │
 │ فخوز   ┆ 14031230 ┆ 12     ┆ 14040824 ┆ … ┆ صادراتی

In [ ]:
dates_desc = sorted(list(all_reports.keys()), reverse=True)

['14041229', '14031230', '14021229', '14011229']

In [ ]:
filtered_all_reports = {}
for fin_year in dates_desc:
    if fin_year in all_reports.keys():
        if all_reports[fin_year]['Operational'][list(all_reports[fin_year]['Operational'].keys())[-1]]['period'] == 12:
            filtered_all_reports[fin_year] = all_reports[fin_year]['Operational'][list(all_reports[fin_year]['Operational'].keys())[-1]]
        else:
            filtered_all_reports[fin_year] = all_reports[fin_year]['Operational'][list(all_reports[fin_year]['Operational'].keys())[-1]]
filtered_all_reports

{'14041229': {'period': '6',
  'publish': 14040824,
  'title': 'اطلاعات و صورت\u200cهای مالی میاندوره\u200cای  دوره ۶ ماهه منتهی به  ۱۴۰۴/۰۶/۳۱ (حسابرسی نشده)(اصلاحیه)',
  'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=A6GHwKf82I6VNWgrotzj7A%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=20'},
 '14031230': {'period': 12,
  'publish': 14040506,
  'title': 'صورت\u200cهای مالی تلفیقی سال مالی منتهی به ۱۴۰۳/۱۲/۳۰ (حسابرسی شده)',
  'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=1z105r5OIg1Yv0h84qhcAw%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=20'},
 '14021229': {'period': 12,
  'publish': 14030420,
  'title': 'صورت\u200cهای مالی تلفیقی سال مالی منتهی به ۱۴۰۲/۱۲/۲۹ (حسابرسی شده)',
  'url': 'https://codal.ir/Reports/Decision.aspx?LetterSerial=gN7ITfhbee1nWKRrszGMMA%3d%3d&rt=0&let=6&ct=0&ft=-1&SheetId=20'},
 '14011229': {'period': 12,
  'publish': 14020418,
  'title': 'صورت\u200cهای مالی تلفیقی سال مالی منتهی به ۱۴۰۱/۱۲/۲۹ (حسابرسی شده)',
  'url': 'https://codal.ir/Reports/Decision.asp

In [ ]:
if dates_desc[0] in all_reports.keys():
    print(all_reports[dates_desc[0]]['Operational'].keys())

dict_keys([14040331, 14040631])


In [ ]:
data.columns = [str(i) for i in range(data.width)]
data = data.insert_column(0, pl.lit(symbol).alias("Symbol"))
data = data.insert_column(1, pl.lit(int(digits.fa_to_en('14030930').replace('/', ''))).alias("Date"))
data = data.insert_column(2, pl.lit(period).alias("Period"))
data = data.insert_column(3, pl.lit(publish).alias("Publish"))
data

Symbol,Date,Period,Publish,0,1,2,3,4,5,6,7
str,i32,i32,i32,str,str,str,str,u32,i64,i64,i64
"""فخوز""",14030930,9,14031020,"""فخوز""","""اسلب""","""تن""","""داخلی""",3,1252457,1031275,202773181
"""فخوز""",14030930,9,14031020,"""فخوز""","""بلوم ، بیلت ""","""تن""","""داخلی""",4,2147637,1494216,202614534
"""فخوز""",14030930,9,14031020,"""فخوز""","""آهن اسفنجی""","""تن""","""داخلی""",5,3537243,215334,120139588
"""فخوز""",14030930,9,14031020,"""فخوز""","""گندله ""","""تن""","""داخلی""",6,5506540,344373,46911941
"""فخوز""",14030930,9,14031020,"""فخوز""","""اسلب""","""تن""","""صادراتی""",10,0,243040,187682558
"""فخوز""",14030930,9,14031020,"""فخوز""","""بلوم ، بیلت ""","""تن""","""صادراتی""",11,0,718167,191041588
"""فخوز""",14030930,9,14031020,"""فخوز""","""آهن اسفنجی""","""تن""","""صادراتی""",12,0,31960,101468179


In [ ]:
symbol = 'فخوز'
period = 9
publish = 14031020
url = 'https://codal.ir/Reports/InterimStatement.aspx?LetterSerial=YQQQaQQQr9ls8uXPQkIRgPeupRng%3d%3d&SheetId=20'
date = 14030930

dict_df = of.create_operational_dataframe(symbol, url, date, period, publish)
df = dict_df['report_this_year']


In [ ]:
df

Symbol,Date,Period,Publish,Product,Unit,Type,Production,Sales,Price
str,i32,i32,i32,str,str,str,i64,i64,i64
"""فخوز""",14030930,9,14031020,"""اسلب""","""تن""","""داخلی""",690087,590616,232053901
"""فخوز""",14030930,9,14031020,"""بلوم ، بیلت ""","""تن""","""داخلی""",1530912,984308,224416890
"""فخوز""",14030930,9,14031020,"""آهن اسفنجی""","""تن""","""داخلی""",3247286,500102,125104491
"""فخوز""",14030930,9,14031020,"""گندله ""","""تن""","""داخلی""",4123197,15661,54727284
"""فخوز""",14030930,9,14031020,"""اسلب""","""تن""","""صادراتی""",0,73346,259861533
"""فخوز""",14030930,9,14031020,"""بلوم ، بیلت ""","""تن""","""صادراتی""",0,473525,219169243
"""فخوز""",14030930,9,14031020,"""آهن اسفنجی""","""تن""","""صادراتی""",0,29390,132843688


In [ ]:
df.to_pandas().to_excel(
    "output.xlsx",
    sheet_name="Sheet1",
    index=False
)

In [ ]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter

out = "output.xlsx"

# --- 1) Create a real workbook with a visible sheet (prevents the error) ---
wb = Workbook()
ws = wb.active
ws.title = "Sheet1"
ws.sheet_state = "visible"
ws.sheet_view.rightToLeft = True
wb.save(out)

titles = ["تولید", "فروش", "نرخ"]
tables = [t1, t2, t3]
gap = 2
excel_row = 1  # 1-based Excel row

with pd.ExcelWriter("output.xlsx", engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    ws = writer.book["Sheet1"]

    for title, table in zip(titles, tables):
        date_row = table.select("Date").to_pandas().iloc[0].tolist()
        pdf = table.drop("Date").to_pandas()

        ncols = pdf.shape[1]
        last_col = get_column_letter(ncols)

        # title (merged)
        ws.merge_cells(f"A{excel_row}:{last_col}{excel_row}")
        ws[f"A{excel_row}"].value = title
        ws[f"A{excel_row}"].font = Font(bold=True)
        ws[f"A{excel_row}"].alignment = Alignment(horizontal="center")

        # date row (between title and header)
        for i, v in enumerate(date_row, start=1):
            ws.cell(row=excel_row + 1, column=i, value=v)

        # table (header + data) under date row
        pdf.to_excel(writer, sheet_name="Sheet1", index=False, startrow=excel_row + 1)

        # advance: title + date + header + data + gap
        excel_row += 3 + len(pdf) + gap
